<a href="https://colab.research.google.com/github/lfmartins/markov-decision-processes/blob/main/Markov_Decision_Processes_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

## Optional installation of Python 3.11
Run the following cell to install and use Python 3.11 in this notebook. As of the time of this writing, the default version of Python in colab is 3.8.

Output is supressed, and installation may take a couple minutes to finish. 

Remove `%%capture` to see the shell messages.

In [5]:
%%capture
!sudo apt-get update -y
!sudo apt-get install python3.11
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2

Check if installation is successful:

In [4]:
!python --version

Python 3.8.10


## Imports

Run the following cell to import required modules and functions.

In [6]:
import numpy as np
import numpy.linalg

# Introduction

We want to build a framework for MDPs. The basic ingredients are:

- A set of *states* $\mathscr{S}$. We assume that $\mathscr{S}$ is finite and let $N=|\mathscr{S}|$
- A set of *actions* $\mathscr{A}$. We assume that $\mathscr{A}$ is finite.
- For each $s\in\mathscr{S}$, a set of *admissible actions for state $s$*, $\mathscr{A}_s\subset\mathscr{A}$.
- For each $s,s'\in\mathscr{S}$ and $a\in\mathscr{A}_s$, a number $p(s'\,|\,s,a)\in[0,1]$.
- For each $s\in\mathscr{S}$ and $a\in\mathscr{A}$, a number $r(s,a)$. We let the set of possible reward be $\mathscr{R}$. We also assume this set to be finite.

We interpret $p(s'\,|\,s,a)$ as the probability that, if the agent is in state $s$ and action $a$ is chosen, the agent will next transition to state $s'$. We require that, for each $s$, $a$ we have:
$$
\sum_{s'\in\mathscr{S}}p(s'\,|\,s,a)=1
$$

The function $r(s,a)$ represents a *reward* received by the agent for visiting state $s$ and choosing action $a$. We can easily extend the definition to cover for randomized rewards.

We will only consider infinite-horizon problems, so we only have to deal with stationary policies. A *randomized policy* is a specification, for every $s\in\mathscr{S}$ and $a\in\mathscr{A}$ of a number $\pi(a\,|\,s)$, interpreted as the probability that action $a$ is chosen when in state $s$. We require:
$$
0\le\pi(a\,|\,s)\le 1,\quad \sum_{a\in\mathscr{A_s}}\pi(a\,|\,s)=1
$$
If, for all $s\in\mathscr{S}$ there is an $a=a(s)\in\mathscr{A}_s$ such that $\pi(a\,|\,s)=1$, we say that $\pi$ is *deterministic*.

The *transition probability matrix* associated to a policy $\pi$ is a $N\times N$ matrix $Q$ defined by:
$$
Q_{ss'}=\sum_{a\in\mathscr{A}_s}\pi(a\,|\,s)p(s'\,|\,s,a)
$$
It is easy to see that this is a stochastic matrix. Thus, if $\pi_0$ is a probability distribution on $\mathscr{S}$, there is a Markov chain $\{(S_t,A_t,R_t)\}_{t\ge 0}\subset \mathscr{S}\times\mathscr{A}\times\mathscr{R}$ such that:

- $\mathbb{P}_{\pi,\pi_0}\left[S_t=s'\,|\,S_{t-1}=s, A_{t-1}=a\right]=p(s'\,|\,s,a)$
- $\mathbb{P}_{\pi,\pi_0}\left[A_t=a\,|\,S_t=s\right]=\pi(a\,|\,s)$ if $a\in\mathscr{A}_t$.
- $R_t=r(S_t,A_t)$

In general, the initial distribution will not be relevant in the formulas below, so we will write simply $P_\pi$ to the probability measure corresponding to the Markov chain. We denote the corresponding expected value by $E_\pi$.

Let $0< \gamma < 1$ we define the *value function* of policy $\pi$ as:
$$
V_\pi(s)=E_\pi\left[\sum_{t=0}^\infty\gamma^tR_t\,|\,S_0=s\right]
$$

This value function is a solution to *Bellman's Equation*:
$$
V_\pi(s)=\sum_{a\in\mathscr{A}_s}\pi(a\,|\,s)\left[r(a,s)+\sum_{s'\in\mathscr{S}}\gamma p(s'\,|\,s,a)V_\pi(s)\right]
$$
The sum is finite since $\gamma$ is in the interval $(0,1)$.

The *optimal value function* is defined as:
$$
V(s)=\max\left\{V_\pi(s)\,|\,\text{$\pi$ is a randomized policy}\right\}
$$

Bellman's Equation for the optimal value function is:
$$
V^*(s)=\max_{a\in\mathscr{A}_s}\left\{r(a,s)+\sum_{s'\in\mathscr{S}}\gamma p(s'\,|\,s,a)V^*(s)\right\}
$$
The optimal policy is always deterministic, and is found by:
$$
\pi^*(s)=\underset{a\in\mathscr{A}_s}{\arg\max}\left\{r(a,s)+\sum_{s'\in\mathscr{S}}\gamma^tp(s'\,|\,s,a)V^*(s)\right\}
$$
Obviously, we have:
$$
V^*(s)=V_{\pi^*}(s)
$$
for all $s\in\mathscr{S}$.

# Data Structures

We need to store three things:

- The transition probabilities $p(s',s,a)$
- The rewards $r(s,a)$
- The policy

A Markov Decision Process is represented by an object in the class `MDP`. This class records all states and actions. States and actions have unique IDs that are mapped to an integer (according to the order by which they are created). The state and action IDs can be any hashable object.

State IDs are stored in a list `__states`, and action IDs are stored in a list `__actions`. States and actions are added to these lists in the order they are created.

We call the *index* os a state `s` the position where `s` is stored in the list `__states`. Similarly the index of an action `a` is its position in the list `__actions`

For storing the transition probabilities, we use a list `__tp`. Each entry in `__tp` is a dictionary. For each action, this dictionary points to an array containing the transition probabilities.

Notice that there is some waste in this description, and in the future we may move to a more sparse representation.


In [24]:
class MDP(object):
  def __init__(self, states, actions, mdp_data):
    self.__states = states
    self.__nstates = len(self.__states)
    self.__state_index = {s: i for i, s in enumerate(self.__states)}
    self.__actions = actions
    self.__nactions = len(self.__actions)
    self.__action_index = {a: i for i, a in enumerate(self.__actions)}
    
    self.__allowed_actions = [set() for _ in range(self.__nstates)]
    self.__r = np.zeros((self.__nactions, self.__nstates), dtype=np.float64)
    self.__p = np.zeros((self.__nstates, self.__nactions, self.__nstates), dtype=np.float64)
    for (s, a), (r, tp) in mdp_data.items():
      try:
         s_index = self.__state_index[s] 
      except KeyError:
        raise ValueError(f'state {s} does not exist')
      try:
        a_index = self.__action_index[a]
      except KeyError:
        raise ValueError(f'action {a} does not exist')
      if a_index in  self.__allowed_actions[s_index]:
        raise ValueError(f'action {a} repeated for state {s}')
      self.__allowed_actions[s_index].add(a_index)
      if len(tp) != self.__nstates:
        raise ValueError(f'transition probability array has wrong size for state {s}, action {a}')
      self.__p[s_index, a_index, :] = tp
      self.__r[a_index, s_index] = r

  def policy_value(self, policy, gamma):
    b = np.array([policy[i, :].dot(self.__r[:, i]) for i in range(self.__nstates)])
    M = gamma * np.array([
        [policy[i,:].dot(self.__p[i, :, j]) for j in range(self.__nstates)]
        for i in range(self.__nstates)
    ])
    V_pi = np.linalg.solve(np.eye(self.__nstates) - M, b)
    return V_pi
 
  def policy_iteration(self, gamma, tol=1e-5, maxiter=100):
    policy = np.random.randint(self.__nstates, size=self.__nactions)
    b = 
    success = True
    for _ in range(maxiter):


    
  def pretty_print(self):
    for s_index, s in enumerate(self.__states):
      print(f'State {s}:')
      for a_index, a in enumerate(self.__actions):
        print(f'  Action {a}. Reward: {self.__r[a_index, s_index]};',
              f'Transition probabilities: {self.__p[s_index, a_index, :]}')


In [25]:
tiny_robot_states = [1, 2, 3, 4]
tiny_robot_actions = ['A', 'B']
tiny_robot_data = {
  (1, 'A'): (1, [1/3, 2/3,   0,   0]),
  (1, 'B'): (4, [  0, 1/2,   0, 1/2]),
  (2, 'A'): (2, [  0, 1/3, 2/3,   0]),
  (2, 'B'): (3, [1/2,   0, 1/2,   0]),
  (3, 'A'): (3, [  0,   0, 1/3, 2/3]),
  (3, 'B'): (2, [  0, 1/2,   0, 1/2]),
  (4, 'A'): (4, [2/3,   0,   0, 1/3]),
  (4, 'B'): (1, [1/2,   0, 1/2,   0]),
}
tiny_robot_mdp = MDP(tiny_robot_states, tiny_robot_actions, tiny_robot_data)

In [26]:
tiny_robot_mdp.pretty_print()

State 1:
  Action A. Reward: 1.0; Transition probabilities: [0.33333333 0.66666667 0.         0.        ]
  Action B. Reward: 4.0; Transition probabilities: [0.  0.5 0.  0.5]
State 2:
  Action A. Reward: 2.0; Transition probabilities: [0.         0.33333333 0.66666667 0.        ]
  Action B. Reward: 3.0; Transition probabilities: [0.5 0.  0.5 0. ]
State 3:
  Action A. Reward: 3.0; Transition probabilities: [0.         0.         0.33333333 0.66666667]
  Action B. Reward: 2.0; Transition probabilities: [0.  0.5 0.  0.5]
State 4:
  Action A. Reward: 4.0; Transition probabilities: [0.66666667 0.         0.         0.33333333]
  Action B. Reward: 1.0; Transition probabilities: [0.5 0.  0.5 0. ]


For now, we model a policy as an array. Rows are state indexes, columns are action indexes.

In [27]:
policy = np.array([
    [1/2, 1/2],
    [1/4, 3/4],
    [2/3, 1/3],
    [0, 1]
]);
policy   

array([[0.5       , 0.5       ],
       [0.25      , 0.75      ],
       [0.66666667, 0.33333333],
       [0.        , 1.        ]])

In [28]:
tiny_robot_mdp.policy_value(policy, 0.8)

array([11.61247481, 11.87213775, 11.18519875, 10.11906943])